# CONFLUENCE Tutorial: Point-Scale Workflow (Fluxnet Example)

In this notebook demonstrates we will build on our experience with the snow simulations for the previous notebook. We'll simulate vertical processes at a single flux tower and compare our results with the flux tower observations .

In [ ]:
# Import required libraries
import sys
import os
from pathlib import Path
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from datetime import datetime
import contextily as cx
import xarray as xr
import numpy as np

# Add CONFLUENCE to path
confluence_path = Path('../').resolve()
sys.path.append(str(confluence_path))

# Import main CONFLUENCE class
from CONFLUENCE import CONFLUENCE

# Set up plotting style
plt.style.use('default')
%matplotlib inline

## 2. Create Point-Scale Configuration

In [ ]:
# Set directory paths
CONFLUENCE_CODE_DIR = confluence_path
CONFLUENCE_DATA_DIR = Path('/work/comphyd_lab/data/CONFLUENCE_data')  # ← User should modify this path

# Load template configuration
config_template_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_point_template.yaml'

# Read config file
with open(config_template_path, 'r') as f:
    config_dict = yaml.safe_load(f)

# Update paths and settings 
config_dict['CONFLUENCE_CODE_DIR'] = str(CONFLUENCE_CODE_DIR)
config_dict['CONFLUENCE_DATA_DIR'] = str(CONFLUENCE_DATA_DIR)

# Update name and experiment id
config_dict['DOMAIN_NAME'] = 'CA-NS7_fluxnet'
config_dict['EXPERIMENT_ID'] = 'CA-NS7_tutorial'
config_dict['POUR_POINT_COORDS'] = '56.6358/-99.9483'

# Save point-scale configuration to temporary file
temp_config_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_point_notebook.yaml'
with open(temp_config_path, 'w') as f:
    yaml.dump(config_dict, f)

# Initialize CONFLUENCE with the new configuration
confluence = CONFLUENCE(temp_config_path)

# Print summary of the key seetings
print("=== Point-Scale Configuration ===")
print(f"Domain Name: {config_dict['DOMAIN_NAME']}")
print(f"Spatial Mode: {config_dict['SPATIAL_MODE']}")
print(f"Location: {config_dict['POUR_POINT_COORDS']}")
print(f"Period: {config_dict['EXPERIMENT_TIME_START']} to {config_dict['EXPERIMENT_TIME_END']}")
print(f"Forcing Data: {config_dict['FORCING_DATASET']}")

## 3. Setup Project Structure

In [ ]:
# Step 1: Project Initialization
print("=== Step 1: Project Initialization ===")
print("Creating point-scale project structure...")

# Setup project
project_dir = confluence.managers['project'].setup_project()

# Create pour point (in this case, our SNOTEL location)
pour_point_path = confluence.managers['project'].create_pour_point()

# List created directories
print("\nCreated directories:")
for item in sorted(project_dir.iterdir()):
    if item.is_dir():
        print(f"  📁 {item.name}")

## 4. Geospatial Domain Definition - Data acquisition 

In [ ]:
# Acquire attributes
print("Acquiring geospatial attributes for point location...")

print(f"Minimal bounding box: {confluence.config.get('BOUNDING_BOX_COORDS')}")

confluence.managers['data'].acquire_attributes()

## 6. Geospatial Domain Definition - Domain creation

In [ ]:
# Geospatial Domain Definition and Analysis
print("=== Step 2: Geospatial Domain Definition and Analysis ===")

# Define domain
print("\nDefining minimal domain for point-scale simulation...")
watershed_path = confluence.managers['domain'].define_domain()
# Discretize domain (single HRU for point-scale)
print("\nCreating single HRU for point-scale simulation...")
hru_path = confluence.managers['domain'].discretize_domain()

# Check outputs
print("\nDomain definition complete:")
print(f"  - Watershed defined: {watershed_path is not None}")
print(f"  - HRUs created: {hru_path is not None}")

# Verify single HRU
hru_dir = project_dir / 'shapefiles' / 'catchment'
if hru_dir.exists():
    hru_files = list(hru_dir.glob('*.shp'))
    if hru_files:
        hru_gdf = gpd.read_file(hru_files[0])
        print(f"  - Number of HRUs: {len(hru_gdf)} (should be 1 for point-scale)")

## 7. Model Agnostic Data Pre-Processing - Data Acquisition

In [ ]:
# Step 3: Model Agnostic Data Pre-Processing
print("=== Step 3: Model Agnostic Data Pre-Processing ===")

# Process observed data 
print("Processing observed data...")
confluence.managers['data'].process_observed_data()

In [ ]:
# Acquire forcings
print(f"\nAcquiring forcing data for point location...")
print(f"Dataset: {confluence.config['FORCING_DATASET']}")
#confluence.managers['data'].acquire_forcings()

## 8. Model Agnostic Data Pre-Processing - Remapping and zonal statistics

In [ ]:
# Run model-agnostic preprocessing
print("\nRunning model-agnostic preprocessing...")

confluence.managers['data'].run_model_agnostic_preprocessing()

print("\nModel-agnostic preprocessing complete")

## 9. Model Specific - Pre Processing 

In [ ]:
# Step 4: Model Specific Processing and Initialization
print("=== Step 4: Model Specific Processing and Initialization ===")

confluence.managers['model'].preprocess_models()

print("\nModel-specific preprocessing complete")

## 10. Model Specific - Initialisation

In [ ]:
# Run models
print(f"\nRunning {confluence.config['HYDROLOGICAL_MODEL']} for point-scale simulation...")
confluence.managers['model'].run_models()

print("\nPoint-scale model run complete")

## Result visualisation

In [ ]:
# Load the simulated SWE data
sim_path = Path(config_dict['CONFLUENCE_DATA_DIR']) / f"domain_{config_dict['DOMAIN_NAME']}" / "simulations" / config_dict['EXPERIMENT_ID'] / "SUMMA" / f"{config_dict['EXPERIMENT_ID']}_day.nc"

# Check for alternative NetCDF file patterns if not found
if not sim_path.exists():
    print(f"Simulated data not found at {sim_path}")
    print("Checking for alternative NetCDF files...")
    alt_sim_paths = list(Path(config_dict['CONFLUENCE_DATA_DIR']).glob(f"domain_{config_dict['DOMAIN_NAME']}/simulations/{config_dict['EXPERIMENT_ID']}/SUMMA/*.nc"))
    
    if alt_sim_paths:
        sim_path = alt_sim_paths[0]
        print(f"Found alternative simulation data at: {sim_path}")
    else:
        raise FileNotFoundError(f"No simulation results found for experiment {config_dict['EXPERIMENT_ID']}")

# Load simulated data
print(f"Loading simulated data from: {sim_path}")
ds = xr.open_dataset(sim_path)

# Extract scalarSWE and convert to DataFrame
sim_swe = ds['scalarSWE'].to_dataframe().reset_index()
# Assuming first HRU for point-scale simulation
sim_swe = sim_swe[sim_swe['hru'] == 1][['time', 'scalarSWE']]
sim_swe.columns = ['Date', 'SWE']
sim_swe.set_index('Date', inplace=True)
print(f"Simulated data period: {sim_swe.index.min()} to {sim_swe.index.max()}")
print(f"Simulated SWE range: {sim_swe['SWE'].min():.2f} to {sim_swe['SWE'].max():.2f} mm")

# If no observed data, just plot simulated
plt.figure(figsize=(12, 6))
plt.plot(sim_swe.index, sim_swe['SWE'], '-', label='Simulated SWE', color='blue', linewidth=2)
plt.title(f"Simulated SWE at {config_dict['DOMAIN_NAME'].replace('_', ' ').title()}", fontsize=14)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Snow Water Equivalent (mm)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()

# Close the dataset
ds.close()

print("\nSWE visualization complete")

## Summary: Point-Scale Modeling Insights

### Next Steps:

1. **Scale up to lumped basin**: Use calibrated parameters
2. **Compare multiple sites**: Test model transferability
